# Seq2Seq

*Sequence to Sequence*.  
*Encoder-Decoder Model*とも。

文章を入力とし、文章を出力するモデル。

In [1]:
from typing import List
import random

import pandas as pd
import MeCab
import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from torchtext import transforms
from torchtext.vocab import build_vocab_from_iterator
from torchvision.transforms import Compose
from dlprog import train_progress

In [2]:
prog = train_progress()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

### 発想

これまで、RNNを用いて、入力した単語列に続く単語を予測するモデルを作成し、単語の予測を繰り返すことで文章を生成した。

RNNは時系列の情報を保持するために隠れ状態$h_t$を用いる。隠れ状態の初期値$h_0$は0ベクトルとしているが、ここで、何らかの入力データから生成したベクトルを$h_0$として用いることを考える。このとき、上手く学習させれば、その入力に基づいた文章を生成できそう。

例えば、入力を画像をとし、CNNを用いて抽出した特徴量を$h_0$として用いるようにすれば、入力画像に基づいた文章が生成できる。画像のキャプションなどが例に挙げられる。  
学習方法は簡単で、入力画像に対して適切な文章が出力されるように学習させるだけ。隠れ状態を通じてRNNからCNNまで逆伝播を繋げる。

では、入力に文章を用いることはできないだろうか。RNNに文章を入力し、最後に出力された隠れ状態を文章ベクトルとする。これを別のRNNへの入力$h_0$とすれば、入力文に基づいた文章生成が可能になる。  
文章ベクトルを生成するRNNを***Encoder***、文章ベクトルを受け取って出力文を生成するRNNを***Decoder***と呼ぶ。

この発想は翻訳タスクに大きく役立つ。入力と出力に同じ意味を持った異なる言語の文章を設定すれば、入力文と同じ意味を持った文章生成が可能になる。

本章では、入力に日本語文、出力に入力と同じ意味を持つ英語文を設定し、日本語→英語の翻訳を行うモデルを作成する。


---

## データセット

翻訳モデルを作るには、同じ意味を持つ文章が複数の言語でまとまっているデータが必要。このようなデータは対訳コーパスと呼んだりする。  
本章では以下のデータセットを使用する。
- [日英中基本文データ](https://nlp.ist.i.kyoto-u.ac.jp/index.php?%E6%97%A5%E8%8B%B1%E4%B8%AD%E5%9F%BA%E6%9C%AC%E6%96%87%E3%83%87%E3%83%BC%E3%82%BF)

In [3]:
df = pd.read_excel('data/JEC_basic_sentence_v1-3.xls', header=None)
df.columns = ['id', 'japanese', 'english', 'chinese']
print('num of data:', len(df))
df.head()

num of data: 5304


,id,japanese,english,chinese
0,#0001,Xではないかとつくづく疑問に思う,I often wonder if it might be X.,难道不会是X吗，我实在是感到怀疑。
1,#0002,Xがいいなといつも思います,I always think X would be nice.,我总觉得X不错。
2,#0003,それがあるようにいつも思います,It always seems like it is there.,我总觉得那好像是有的。
3,#0004,それが多すぎないかと正直思う,I honestly feel like there is too much.,老实说我觉得那太多了。
4,#0005,山田はみんなに好かれるタイプの人だと思う,I think that Yamada is the type everybody likes.,我想山田是受大家欢迎的那种人。


分かち書き

In [4]:
tagger = MeCab.Tagger('-Owakati')
def tokenize(data: List[str], l='en') -> List[List[str]]:
    if l == 'ja':
        return [tagger.parse(sentence).split() for sentence in data]
    elif l == 'en':
        return [sent.replace('.', ' .').lower().split() for sent in data]

In [5]:
text_ja = tokenize(df['japanese'], l='ja')
text_en = tokenize(df['english'], l='en')

# examples
text_ja[0], text_en[0]

(['X', 'で', 'は', 'ない', 'か', 'と', 'つくづく', '疑問', 'に', '思う'],
 ['i', 'often', 'wonder', 'if', 'it', 'might', 'be', 'x', '.'])


---

## 学習データ

入力と出力のペアを作成する。  
前処理はこれまでと同じ。

In [6]:
pad, bos, eos, unk = '<pad>', '<bos>', '<eos>', '<unk>'
specials = [pad, bos, eos, unk]
vocab_ja = build_vocab_from_iterator(text_ja, specials=specials)
vocab_en = build_vocab_from_iterator(text_en, specials=specials)

# 後で使う
def ids_to_sentence(token_ids, l='en'):
    """ID列 -> 文章"""
    vocab = eval(f'vocab_{l}')
    tokens = []
    for i in token_ids[1:]:
        if i == vocab.get_stoi()[eos]:
            break
        tokens.append(vocab.get_itos()[i])
    return ' '.join(tokens)

In [7]:
transform_ja = Compose([
    transforms.AddToken(eos, begin=False),
    transforms.VocabTransform(vocab_ja),
    transforms.ToTensor(),
])

transform_en = Compose([
    transforms.AddToken(bos, begin=True),
    transforms.AddToken(eos, begin=False),
    transforms.VocabTransform(vocab_en),
    transforms.ToTensor(),
])

# 語彙数
n_vocab_ja = len(vocab_ja)
n_vocab_en = len(vocab_en)
n_vocab_ja, n_vocab_en

(6402, 6123)

入力文と出力文のペアを作成する。  

Decoder（出力文を生成するRNN）への入力も用意する必要がある。出力文の頭に\<BOS>を付与したものとする。

例）
Encoderへの入力（入力文） | Decoderへの入力 | Eecoderの出力（出力文）
--- | --- | ---
夏 休み が 終わり ました 。 \<EOS> | \<BOS> Summer vacation is over . | Summer vacation is over . \<EOS>
ツイッター は 亡くなり ました 。 \<EOS> | \<BOS> Twitter is dead . | Twitter is dead . \<EOS>
今日 から X で 暮らし ましょう 。 \<EOS> | \<BOS> Let 's live in X from today . | Let 's live in X from today . \<EOS>

DataLoaderを作成する

In [8]:
class TextDataset(Dataset):
    def __init__(self, in_text, out_text, in_transform, out_transform):
        # 前処理
        self.in_text = [in_transform(text) for text in in_text]
        self.out_text = [out_transform(text) for text in out_text]
        self.n_samples = len(in_text)

    def __getitem__(self, index):
        in_text = self.in_text[index]
        out_text = self.out_text[index]
        return in_text, out_text[:-1], out_text[1:]

    def __len__(self):
        return self.n_samples

# パディング
def to_padded_tensor(text_data: List[int], pad_value: int = 0) -> torch.Tensor:
    data = pad_sequence(text_data, batch_first=True, padding_value=pad_value)
    return data

# バッチ内の系列長を揃える
def collate_fn(batch):
    x_enc, x_dec, y_dec = zip(*batch)
    x_enc = to_padded_tensor(x_enc)
    x_dec = to_padded_tensor(x_dec)
    y_dec = to_padded_tensor(y_dec)
    return x_enc, x_dec, y_dec

dataset = TextDataset(text_ja, text_en, transform_ja, transform_en)
dataloader = DataLoader(
    dataset,
    batch_size=32,
    shuffle=True,
    collate_fn=collate_fn
)

# examples
x_enc, x_dec, y_dec = next(iter(dataloader))
print('x_enc:', x_enc.shape)
print('x_dec:', x_dec.shape)
print('y_dec:', y_dec.shape)

x_enc: torch.Size([32, 18])
x_dec: torch.Size([32, 17])
y_dec: torch.Size([32, 17])



---

## モデル構築

モデルを作るよん。

### Encoder

入力文を入れて隠れ状態を出力するだけのRNN。RNN層と線形層で作る。

RNN層からの隠れ状態は、最後の単語を入力した時点のものを使用する。最後のトークンではないので注意。\<EOS>が入力された時点の隠れ状態を使用する。

In [9]:
eos_id = vocab_ja.get_stoi()[eos]

class Encoder(nn.Module):
    def __init__(self, n_vocab, embed_size, hidden_size):
        super().__init__()
        self.embedding = nn.Embedding(n_vocab, embed_size)
        self.rnn = nn.RNN(embed_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, hidden_size)

    def forward(self, x):
        eos_positions = x == eos_id
        x = self.embedding(x)
        y, _ = self.rnn(x)
        h = y[eos_positions] # (batch_size, hidden_size)
        h = self.fc(h).unsqueeze(0) # (1, batch_size, hidden_size)
        return h

### Decoder

Encoderから出力された隠れ状態を受け取り、出力文を生成するRNN。Encoder同様、RNN層と線形層で作る。



In [10]:
class Decoder(nn.Module):
    def __init__(self, n_vocab, embed_size, hidden_size):
        super().__init__()
        self.embedding = nn.Embedding(n_vocab, embed_size)
        self.rnn = nn.RNN(embed_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, n_vocab)

    def forward(self, x, h):
        x = self.embedding(x)
        y, h = self.rnn(x, h)
        y = self.fc(y)
        return y, h

### Seq2Seq

EncoderとDecoderを合わせて、入力から出力までの一連の処理を行うモデルを作る。

In [11]:
class Seq2Seq(nn.Module):
    def __init__(self, n_in_vocab, n_out_vocab, embed_size, hidden_size):
        super().__init__()
        self.encoder = Encoder(n_in_vocab, embed_size, hidden_size)
        self.decoder = Decoder(n_out_vocab, embed_size, hidden_size)

    def forward(self, x_enc, x_dec):
        h = self.encoder(x_enc)
        y, _ = self.decoder(x_dec, h)
        return y


---

## 学習

学習するよん。特に変わったことはしないよ。

In [12]:
criterion = nn.CrossEntropyLoss(ignore_index=vocab_en[pad])
def train(model, optimizer, n_epochs, prog_unit=1):
    model.train()
    prog.start(n_iter=len(dataloader), n_epochs=n_epochs, unit=prog_unit)
    for _ in range(n_epochs):
        for x_enc, x_dec, y_dec in dataloader:
            optimizer.zero_grad()
            x_enc = x_enc.to(device)
            x_dec = x_dec.to(device)
            y_dec = y_dec.to(device).ravel()

            y_pred = model(x_enc, x_dec).reshape(-1, n_vocab_en)
            loss = criterion(y_pred, y_dec)
            loss.backward()
            optimizer.step()
            prog.update(loss.item())

In [13]:
model = Seq2Seq(len(vocab_ja), len(vocab_en), 1024, 1024).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [14]:
train(model, optimizer, 100, 10)

   1-10/100: ######################################## 100% [00:00:23.89] loss: 4.11958 
  11-20/100: ######################################## 100% [00:00:23.29] loss: 2.32198 
  21-30/100: ######################################## 100% [00:00:23.26] loss: 1.22940 
  31-40/100: ######################################## 100% [00:00:23.54] loss: 0.58921 
  41-50/100: ######################################## 100% [00:00:23.30] loss: 0.25225 
  51-60/100: ######################################## 100% [00:00:23.48] loss: 0.10013 
  61-70/100: ######################################## 100% [00:00:23.53] loss: 0.03445 
  71-80/100: ######################################## 100% [00:00:23.58] loss: 0.05284 
  81-90/100: ######################################## 100% [00:00:23.85] loss: 0.00775 
 91-100/100: ######################################## 100% [00:00:23.87] loss: 0.00394 



---

## 翻訳

作成したモデルに日本語文を入力し、英語に翻訳して出力する。  
未知語は非対応。

In [15]:
@torch.no_grad()
def translate(model, text, max_len=100, decisive=True):
    model.eval()
    tokens = tokenize([text], 'ja')[0]
    tokens = transform_ja(tokens).unsqueeze(0).to(device)
    h = model.encoder(tokens)
    next_token = vocab_en[bos]

    tokens = []
    for _ in range(max_len):
        next_token = torch.tensor(next_token).reshape(1, 1).to(device)
        y, h = model.decoder(next_token, h)
        y = F.softmax(y.ravel(), dim=0)

        if decisive:
            next_token = y.argmax().item() # 決定的な出力
        else:
            next_token = random.choices(range(len(y)), weights=y)[0] # 確率的な出力

        if next_token == vocab_en[eos]:
            break
        tokens.append(next_token)
    return ' '.join([vocab_en.get_itos()[t] for t in tokens])

まずは学習データに含まれているものから。

In [23]:
n = 3
for _ in range(n):
    i = random.randint(0, len(df))
    sentence = df['japanese'][i]
    answer = df['english'][i]
    print('input:', sentence)
    print('output:', translate(model, sentence))
    print('answer:', answer)
    print()

input: Xを消費者が可能にする
output: consumers will enable x .
answer: Consumers will enable X.

input: 専門スタッフがあなたの健康づくりを手伝います
output: our specialized staff will help you healthy .
answer: Our specialized staff will help you healthy.

input: 彼が数のしくみを理解します
output: he understands the mechanism of the number .
answer: He understands the mechanism of the number.



学習データにない適当な文章も試してみる。

In [29]:
sentences = [
    '私は猫です。',
    '昨日はいい天気でしたね。',
    '彼は健康な身体を持っています。'
]

for sentence in sentences:
    print('input:', sentence)
    print('output:', translate(model, sentence))
    print()

input: 私は猫です。
output: i did x .

input: 昨日はいい天気でしたね。
output: mr . yamada is continuing on from the last time .

input: 彼は健康な身体を持っています。
output: he suddenly looks out .



めちゃくちゃね。